In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import seaborn as sns

sns.set()

In [ ]:
df =  pd.read_excel('../1_Data/datasets/2022_smd_hourly.xlsx' ,sheet_name='ISO NE CA')
df['Date'] = df.Date + pd.to_timedelta(df.Hr_End, unit='h')
df.sort_values('Date', inplace=True)

train_size = int(len(df) * 0.91)
df = df.iloc[:train_size]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Dropout
from keras.optimizers import Adam


features_considered = ['DA_LMP', 'DA_EC', 'DA_MLC', 'DA_Demand', 'Dry_Bulb', 'RT_Demand', 'System_Load'] 
features = df[features_considered]
features.index = df.index  

In [ ]:
scaler_features = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler_features.fit_transform(features)

target = df['RT_LMP'].values.reshape(-1, 1)
scaler_target = MinMaxScaler(feature_range=(0, 1))
scaled_target = scaler_target.fit_transform(target)

look_back = 24  
batch_size = 50  

generator = TimeseriesGenerator(scaled_features, scaled_target,
                                length=look_back, batch_size=batch_size)

model = Sequential([
    LSTM(100, activation='tanh', return_sequences=True, input_shape=(look_back, len(features_considered))),
    Dense(1)
])
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

model.fit_generator(generator, epochs=10)

last_24_hours_features = scaled_features[-look_back:]  
last_24_hours_features = last_24_hours_features.reshape((1, look_back, len(features_considered)))

predicted_demand_scaled = model.predict(last_24_hours_features)

predicted_demand = scaler_target.inverse_transform(predicted_demand_scaled)

# Predict the Next 24 Hours

In [ ]:
input_features = last_24_hours_features.copy()

predicted_demand_scaled = []

for i in range(24):  
    current_pred = model.predict(input_features)
    predicted_demand_scaled.append(current_pred[0, 0])  
    
    new_input = np.roll(input_features, -1, axis=1)  
    new_input[0, -1, :] = current_pred  
    
    input_features = new_input  

predicted_demand_scaled = np.array(predicted_demand_scaled).reshape(-1, 1)
predicted_demand = scaler_target.inverse_transform(predicted_demand_scaled)

In [ ]:
actual_demand = df['RT_LMP'][-24:].values  

plt.figure(figsize=(10, 6))
plt.plot(range(24), actual_demand, label='Actual Price', marker='o')
plt.plot(range(24), predicted_demand.flatten(), label='Forecasted Price', marker='x', linestyle='--')
plt.title('24-Hour Forecast vs Actuals')
plt.xlabel('Hours from Now')
plt.ylabel('LMP')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(actual_demand, predicted_demand)
mse = mean_squared_error(actual_demand, predicted_demand)
rmse = np.sqrt(mse)
r_squared = r2_score(actual_demand, predicted_demand)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r_squared}")